In [1]:
import sys, os
sys.path.append('..')
from utils_ft import get_model_path, get_output_dir, get_root_path, to_relative
from scripts.train_mlx import run_mlx_training, fuse_model
import json
import matplotlib.pyplot as plt
import pandas as pd

# Config
MODEL_TYPE = "qwen"
#Model_TYPE = "llama" 
base_model = get_model_path(MODEL_TYPE)
data_folder = os.path.join(get_root_path(), "Finetuning", "data")
adapter_path = os.path.join(get_root_path(), "Finetuning", "adapters", MODEL_TYPE)
os.makedirs(adapter_path, exist_ok=True)
fused_output = get_output_dir(MODEL_TYPE, "mlx")

In [2]:
run_mlx_training(base_model, data_folder, adapter_path, iters=300)

--- Initializing MLX Fine-tuning ---
--- Model: ./local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16 ---
--- Data Folder: ./Finetuning/data ---
Executing MLX LoRA training command...
Loading pretrained model


The tokenizer you are loading from '/Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading datasets
Training
Trainable parameters: 0.151% (11.534M/7615.617M)
Starting training..., iters: 300


Calculating loss...: 100%|██████████| 11/11 [00:11<00:00,  1.05s/it]


Iter 1: Val loss 1.297, Val took 11.589s


Calculating loss...: 100%|██████████| 11/11 [00:11<00:00,  1.05s/it]


Iter 10: Val loss 0.693, Val took 11.568s
Iter 10: Train loss 0.948, Learning Rate 5.000e-05, It/sec 0.590, Tokens/sec 421.859, Trained Tokens 7151, Peak mem 25.695 GB


Calculating loss...:  73%|███████▎  | 8/11 [00:09<00:03,  1.14s/it]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/mlx_lm/__main__.py", line 30, in <module>
    submodule.main()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/mlx_lm/lora.py", line 362, in main
    run(types.SimpleNamespace(**args))
  File "/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/mlx_lm/lora.py", line 334, in run
    train_model(args, model, train_set, valid_set, training_callback)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/mlx_lm/lora.py", line 288, in train_model
    train(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/mlx_lm/tuner/trainer.py", line 274, in train
    val_

KeyboardInterrupt: 

In [ ]:
def plot_training_results(log_dir):
    report_path = os.path.join(log_dir, "report.json")
    
    if not os.path.exists(report_path):
        print(f"Error: report.json not found in ./{to_relative(log_dir)}")
        return

    with open(report_path, "r") as f:
        log_data = json.load(f)

    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(log_data)

    # Detect format: MLX uses 'train_loss', HF uses 'loss'
    train_col = 'train_loss' if 'train_loss' in df.columns else 'loss'
    val_col = 'val_loss' if 'val_loss' in df.columns else 'eval_loss'
    iter_col = 'iteration' if 'iteration' in df.columns else 'step'

    # Filter out empty rows for specific metrics
    train_df = df[df[train_col].notna()]
    val_df = df[df[val_col].notna()]

    plt.figure(figsize=(10, 5))
    
    if not train_df.empty:
        plt.plot(train_df[iter_col], train_df[train_col], label='Training Loss', alpha=0.7)
    
    if not val_df.empty:
        plt.plot(val_df[iter_col], val_df[val_col], label='Validation Loss', color='red', marker='x')

    plt.title('ASP Autoformalization Fine-tuning Progress')
    plt.xlabel('Iterations / Steps')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# Run the visual inspection
# For MLX, point to the adapter folder. For PyTorch, point to the output folder.
plot_training_results(adapter_path)

No log file found at /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/adapters/qwen/report.json. Check if training completed.


In [3]:
fuse_model(base_model, adapter_path, fused_output)
print(f"Success! Model ready at {fused_output}")

Fusing LoRA adapters into base model...
--- Target Path: ./local_models/ft_qwen_mlx ---
Loading pretrained model


The tokenizer you are loading from '/Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Success! Fused model saved to: ./local_models/ft_qwen_mlx
Success! Model ready at /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/ft_qwen_mlx
